If you've ever tried to understand how [LangChain's ReAct agent](https://christianjmills.com/series/notes/mastering-llms-course-notes.html) works under the hood, you've probably found yourself drowning in layers of abstractions — executors, runnables, callbacks, and chains. It's powerful, but far from obvious what's actually happening.

One question that puzzled me: **how does streaming work with tool calls?** In a single response, the model might stream text tokens AND request a tool call. How do we handle both at once?

Looking at the [OpenAI function calling docs](https://platform.openai.com/docs/guides/function-calling?api-mode=chat), the answer becomes clear. When streaming, tool calls arrive as deltas — first the function name and ID, then arguments piece by piece:

```
[{"index": 0, "id": "call_abc123", "function": {"arguments": "", "name": "get_weather"}, "type": "function"}]
[{"index": 0, "id": null, "function": {"arguments": "{\"", "name": null}, "type": null}]
[{"index": 0, "id": null, "function": {"arguments": "location", "name": null}, "type": null}]
...
```

So our streaming strategy is:
- **Text content**: stream immediately as tokens arrive
- **Tool calls**: accumulate argument chunks until complete, then execute

Let's build a simple ReAct agent from scratch that handles both.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "..."
os.environ["HTTP_PROXY"] = None

## Message Schemas

In [2]:
from pydantic import BaseModel
from typing import Literal, Callable, Any
import json


class ToolCall(BaseModel):
    id: str
    name: str
    arguments: str


class Message(BaseModel):
    role: Literal["system", "user", "assistant", "tool"]
    content: str | None = None
    tool_calls: list[ToolCall] | None = None
    tool_call_id: str | None = None

    def to_dict(self) -> dict:
        d = {"role": self.role}
        if self.content is not None:
            d["content"] = self.content
        if self.tool_calls:
            d["tool_calls"] = [
                {"id": tc.id, "type": "function", "function": {"name": tc.name, "arguments": tc.arguments}}
                for tc in self.tool_calls
            ]
        if self.tool_call_id:
            d["tool_call_id"] = self.tool_call_id
        return d

## Tool Schema

In [3]:
class Tool(BaseModel):
    name: str
    description: str
    parameters: dict
    fn: Callable[..., Any]

    class Config:
        arbitrary_types_allowed = True

    def to_openai_schema(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": self.parameters,
            },
        }

    def execute(self, arguments: str) -> str:
        args = json.loads(arguments)
        result = self.fn(**args)
        return str(result)

/tmp/ipykernel_2024579/1257762879.py:1: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class Tool(BaseModel):


## Agent

In [4]:
from openai import OpenAI
from typing import Generator
import httpx


class Agent:
    def __init__(self, tools: list[Tool], model: str = "gpt-4o-mini"):
        self.tools = {tool.name: tool for tool in tools}
        self.model = model
        self.client = OpenAI(
            http_client=httpx.Client(
                timeout=10.0,
                proxy=os.environ.get("HTTP_PROXY")
            )
        )

    def _get_tools_schema(self) -> list[dict]:
        return [tool.to_openai_schema() for tool in self.tools.values()]

    def run(self, messages: list[Message]) -> list[Message]:
        msgs = [m.to_dict() for m in messages]
        new_messages: list[Message] = []

        while True:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=msgs,
                tools=self._get_tools_schema() if self.tools else None,
            )

            choice = response.choices[0]
            assistant_msg = choice.message

            if not assistant_msg.tool_calls:
                final_msg = Message(role="assistant", content=assistant_msg.content)
                new_messages.append(final_msg)
                return new_messages

            tool_calls = [
                ToolCall(id=tc.id, name=tc.function.name, arguments=tc.function.arguments)
                for tc in assistant_msg.tool_calls
            ]
            assistant_tool_msg = Message(role="assistant", tool_calls=tool_calls)
            new_messages.append(assistant_tool_msg)
            msgs.append(assistant_tool_msg.to_dict())

            for tc in tool_calls:
                tool = self.tools[tc.name]
                result = tool.execute(tc.arguments)
                tool_result_msg = Message(role="tool", content=result, tool_call_id=tc.id)
                new_messages.append(tool_result_msg)
                msgs.append(tool_result_msg.to_dict())

    def run_stream(self, messages: list[Message]) -> Generator[str | Message, None, None]:
        msgs = [m.to_dict() for m in messages]

        while True:
            stream = self.client.chat.completions.create(
                model=self.model,
                messages=msgs,
                tools=self._get_tools_schema() if self.tools else None,
                stream=True,
            )

            content_buffer = ""
            tool_calls_buffer: dict[int, ToolCall] = {}

            for chunk in stream:
                delta = chunk.choices[0].delta

                if delta.content:
                    content_buffer += delta.content
                    yield delta.content

                if delta.tool_calls:
                    for tc_delta in delta.tool_calls:
                        idx = tc_delta.index
                        if idx not in tool_calls_buffer:
                            tool_calls_buffer[idx] = ToolCall(
                                id=tc_delta.id or "",
                                name=tc_delta.function.name or "",
                                arguments="",
                            )
                        if tc_delta.function.arguments:
                            tool_calls_buffer[idx].arguments += tc_delta.function.arguments

            if not tool_calls_buffer:
                yield Message(role="assistant", content=content_buffer)
                return

            tool_calls = list(tool_calls_buffer.values())
            assistant_tool_msg = Message(role="assistant", content=content_buffer or None, tool_calls=tool_calls)
            yield assistant_tool_msg
            msgs.append(assistant_tool_msg.to_dict())

            for tc in tool_calls:
                tool = self.tools[tc.name]
                result = tool.execute(tc.arguments)
                tool_result_msg = Message(role="tool", content=result, tool_call_id=tc.id)
                yield tool_result_msg
                msgs.append(tool_result_msg.to_dict())

## Usage Example

In [5]:
def get_weather(location: str) -> str:
    return f"Weather in {location}: 22°C, sunny"


weather_tool = Tool(
    name="get_weather",
    description="Get current weather for a location",
    parameters={
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "City name"},
        },
        "required": ["location"],
    },
    fn=get_weather,
)

agent = Agent(tools=[weather_tool])

In [6]:
history = [Message(role="user", content="What's the weather in Tokyo?")]
new_msgs = agent.run(history)
history.extend(new_msgs)

for msg in history:
    print(f"{msg.role}: {msg.content or msg.tool_calls}")

user: What's the weather in Tokyo?
assistant: [ToolCall(id='call_bAGWiKKVRnhmR2hkYh9JNMiE', name='get_weather', arguments='{"location":"Tokyo"}')]
tool: Weather in Tokyo: 22°C, sunny
assistant: The weather in Tokyo is currently 22°C and sunny.


In [8]:
history = [Message(role="user", content="What's the weather in Paris? Give me detailed recommendations on what to wear.")]

for item in agent.run_stream(history):
    if isinstance(item, str):
        print(item, end="", flush=True)
    else:
        history.append(item)
        if item.tool_calls:
            print(f"\nCalling: {item.tool_calls[0].name}({item.tool_calls[0].arguments})")
        elif item.role == "tool":
            print(f"Result: {item.content}\n")


Calling: get_weather({"location":"Paris"})
Result: Weather in Paris: 22°C, sunny

The current weather in Paris is 22°C and sunny. Here are some detailed recommendations on what to wear:

1. **Top**: A lightweight, breathable shirt or blouse would be ideal. You can opt for short sleeves or a light long-sleeve option for sun protection.

2. **Bottoms**: Comfortable trousers or shorts are great choices. Consider using lightweight materials that will keep you cool while walking around the city.

3. **Footwear**: Comfortable walking shoes or sandals are essential since you'll want to explore and possibly walk a lot.

4. **Accessories**: A stylish hat can help shield your face from the sun, and sunglasses are a must. 

5. **Outerwear**: While the weather is warm, it’s good to have a light cardigan or jacket handy in case of a breeze, especially in the evening.

6. **Sun Protection**: Don't forget sunscreen! Even if it’s not extremely hot, protecting your skin from UV rays is important.

Enj